<a href="https://colab.research.google.com/github/giholv/TechWeather/blob/main/spark_interrup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# instalar as dependências
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [5]:
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [6]:
get_ipython().system_raw('./ngrok authtoken 2ltM8DyTSfaSjG7gfmCWpvvuB88_6Zs1qpMZSM7SkkNarfeYV')
get_ipython().system_raw('./ngrok http 4050 &')

In [7]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[],"uri":"/api/tunnels"}


In [8]:
spark

In [9]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/datainterrup'
# Ler o arquivo CSV com codificação UTF-8
interrup = spark.read.option("encoding", "ISO-8859-1").csv(path,sep= ";", inferSchema=True,header=True)

Mounted at /content/drive


In [10]:
interrup.limit(10).toPandas()

,DatGeracaoConjuntoDados,IdeConjuntoUnidadeConsumidora,DscConjuntoUnidadeConsumidora,DscAlimentadorSubestacao,DscSubestacaoDistribuicao,NumOrdemInterrupcao,DscTipoInterrupcao,IdeMotivoInterrupcao,DatInicioInterrupcao,DatFimInterrupcao,DscFatoGeradorInterrupcao,NumNivelTensao,NumUnidadeConsumidora,NumConsumidorConjunto,NumAno,NomAgenteRegulado,SigAgente,NumCPFCNPJ
0,2024-03-14,12590,TAQUARI,TQ-TQ303,TQ,2019-12,Não Programada,0,2019-01-01 00:00:00,2019-01-01 08:00:00,INTERNA;NAO PROGRAMADA;MEIO AMBIENTE;ANIMAIS,13800,101,50241,2019,ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A,ELETROACRE,04065033000170
1,2024-03-14,12594,Epitaciolândia,SEXP-XP201,SEXP,2019-29702,Não Programada,0,2019-02-19 18:22:00,2019-02-19 21:46:00,INTERNA;NAO PROGRAMADA;PROPRIAS DO SISTEMA;FAL...,127,1,24160,2019,ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A,ELETROACRE,04065033000170
2,2024-03-14,12590,TAQUARI,SEAC-AC202,SEAC,2019-58586,Não Programada,0,2019-04-09 05:55:00,2019-04-09 08:37:59,INTERNA;NAO PROGRAMADA;MEIO AMBIENTE;ARVORE OU...,13800,3,50524,2019,ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A,ELETROACRE,04065033000170
3,2024-03-14,12596,São Francisco,SESQ SQ221,SESQ,83743,Não Programada,0,2019-06-06 17:16:57,2019-06-06 17:43:59,INTERNA - NAO PROGRAMADA - PROPRIAS DO SISTEMA...,127,32,48963,2019,ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A,ELETROACRE,04065033000170
4,2024-03-14,12596,São Francisco,SESQ-SQ207,SESQ,2019-110233,Não Programada,0,2019-08-10 16:41:48,2019-08-10 18:33:59,INTERNA - NAO PROGRAMADA - PROPRIAS DO SISTEMA...,127,1,48826,2019,ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A,ELETROACRE,04065033000170
5,2024-03-14,12592,Tarauacá,USTAR-003,USTAR,2019-138552,Não Programada,0,2019-09-27 07:41:45,2019-09-27 09:09:00,INTERNA;NAO PROGRAMADA;PROPRIAS DO SISTEMA;FAL...,13800,40,8513,2019,ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A,ELETROACRE,04065033000170
6,2024-03-14,12593,Sena Madureira,SESN-SN202,SESN,2019-163999,Não Programada,0,2019-11-12 03:38:55,2019-11-12 06:20:59,INTERNA;NAO PROGRAMADA;PROPRIAS DO SISTEMA;NAO...,13800,583,14849,2019,ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A,ELETROACRE,04065033000170
7,2024-03-14,12593,Sena Madureira,SESN-SN202,SESN,2019-19370,Não Programada,0,2019-02-04 13:10:00,2019-02-04 14:10:00,INTERNA;NAO PROGRAMADA;MEIO AMBIENTE;DESCARGA ...,13800,52,14547,2019,ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A,ELETROACRE,04065033000170
8,2024-03-14,12597,Tangará,SETG-TG206,SETG,2019-48372,Não Programada,0,2019-03-25 14:25:00,2019-03-26 14:00:00,INTERNA;NAO PROGRAMADA;MEIO AMBIENTE;DESCARGA ...,13800,7,64577,2019,ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A,ELETROACRE,04065033000170
9,2024-03-14,12595,Cruzeiro do Sul,USCDS-003,USCDS,2019-75855,Não Programada,0,2019-05-15 17:56:43,2019-05-15 19:38:00,INTERNA;NAO PROGRAMADA;PROPRIAS DO SISTEMA;FAL...,127,1,39359,2019,ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A,ELETROACRE,04065033000170


In [11]:
from pyspark.sql import functions as f
from pyspark.sql.types import StringType, DoubleType, DateType,TimestampType
import pandas as pd
from pyspark.sql.functions import date_format

spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

# Converter as colunas de data para o formato correto
interrup = interrup.withColumn('DatInicioInterrupcao', f.to_date(f.col('DatInicioInterrupcao'), 'yyyy-MM-dd HH:mm:ss'))
interrup = interrup.withColumn('DatFimInterrupcao', f.to_date(f.col('DatFimInterrupcao'), 'yyyy-MM-dd HH:mm:ss'))

interrup = interrup.withColumn('NumAno', f.col('NumAno').cast(StringType()))

# Converter o timestamp para string apenas com a data 'yyyy-MM-dd'

interrup = interrup.withColumn('DatInicioInterrupcao', date_format(f.col('DatInicioInterrupcao'), 'yyyy/MM/dd'))

In [12]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 8.2 MB/s eta 0:00:00


In [13]:
from pyspark.sql.types import StringType, TimestampType
from pyspark.sql.functions import udf, col, regexp_replace, when, upper, split
from unidecode import unidecode

# Função para remover acentuação usando unidecode
def remove_accents(input_str):
    if input_str:
        return unidecode(input_str)
    return None

#função para remover acentos
remove_accents_udf = udf(remove_accents, StringType())

# Aplicar a função para remover acentuação na mesma coluna
interrup = interrup.withColumn("DscConjuntoUnidadeConsumidora",
                               when(col("DscConjuntoUnidadeConsumidora").isNotNull(),
                                    remove_accents_udf(col("DscConjuntoUnidadeConsumidora"))).otherwise(None))

# Remover caracteres especiais na mesma coluna
interrup = interrup.withColumn("DscConjuntoUnidadeConsumidora",
    regexp_replace(col("DscConjuntoUnidadeConsumidora"), "[^a-zA-Z0-9 ]", ""))

# Remover espaços em branco extras na mesma coluna
interrup = interrup.withColumn("DscConjuntoUnidadeConsumidora",
    regexp_replace(col("DscConjuntoUnidadeConsumidora"), "\s+", " "))

# Remover espaços em branco no início e no final na mesma coluna
interrup = interrup.withColumn("DscConjuntoUnidadeConsumidora",
    regexp_replace(col("DscConjuntoUnidadeConsumidora"), "^\s+|\s+$", ""))

# Converte a coluna para caixa alta

interrup = interrup.withColumn("DscConjuntoUnidadeConsumidora", upper(col("DscConjuntoUnidadeConsumidora")))
interrup = interrup.withColumn("DscFatoGeradorInterrupcao", upper(col("DscFatoGeradorInterrupcao")))
interrup = interrup.withColumn("NomAgenteRegulado", upper(col("NomAgenteRegulado")))

interrup = interrup.withColumn("DscTipoInterrupcao", upper(col("DscTipoInterrupcao")))
interrup = interrup.withColumn("DscTipoInterrupcao",
                               when(col("DscTipoInterrupcao").isNotNull(),
                                    remove_accents_udf(col("DscTipoInterrupcao"))).otherwise(None))

#remove acentuação e caracteres
interrup = interrup.withColumn("DscTipoInterrupcao",
    regexp_replace(col("DscTipoInterrupcao"), "[^a-zA-Z0-9 ]", ""))

#substitui o separador - por ;
interrup = interrup.withColumn("DscFatoGeradorInterrupcao",
    regexp_replace(col("DscFatoGeradorInterrupcao"), " - ", ";"))

#separa a coluna descrição dos fato gerador das interrupções em 4 partes

interrup\
        .where('NumCPFCNPJ == 61695227000193')
interrup = interrup.withColumn("parte_1", split(interrup['DscFatoGeradorInterrupcao'], ";").getItem(0)) \
             .withColumn("parte_2", split(interrup['DscFatoGeradorInterrupcao'], ";").getItem(1)) \
             .withColumn("parte_3", split(interrup['DscFatoGeradorInterrupcao'], ";").getItem(2)) \
             .withColumn("parte_4", split(interrup['DscFatoGeradorInterrupcao'], ";").getItem(3))

#substitui valores em 'None' para desconhecido
interrup = interrup.fillna({'parte_4':'DESCONHECIDO'})

#remove as colunas data de geração do conjunto e descrição do alimentador
interrup = interrup.drop('DatGeracaoConjuntoDados')
interrup = interrup.drop('DscAlimentadorSubestacao')

#substitui valores
interrup = interrup.withColumn('DscConjuntoUnidadeConsumidora',\
                               regexp_replace('DscConjuntoUnidadeConsumidora', 'ALEX DE GUSMAO', 'ALEXANDRE DE GUSMAO'))\
                    .withColumn('DscConjuntoUnidadeConsumidora',\
                               regexp_replace('DscConjuntoUnidadeConsumidora', 'MIGUEL REALERETICULADO', 'MIGUEL REALE'))\
                    .withColumn('DscConjuntoUnidadeConsumidora',\
                               regexp_replace('DscConjuntoUnidadeConsumidora', 'BANDEIRANTESRETICULADO','BANDEIRANTES'))
interrup\
    .where('NumCPFCNPJ == 61695227000193')

DataFrame[IdeConjuntoUnidadeConsumidora: string, DscConjuntoUnidadeConsumidora: string, DscSubestacaoDistribuicao: string, NumOrdemInterrupcao: string, DscTipoInterrupcao: string, IdeMotivoInterrupcao: int, DatInicioInterrupcao: string, DatFimInterrupcao: date, DscFatoGeradorInterrupcao: string, NumNivelTensao: int, NumUnidadeConsumidora: int, NumConsumidorConjunto: int, NumAno: string, NomAgenteRegulado: string, SigAgente: string, NumCPFCNPJ: string, parte_1: string, parte_2: string, parte_3: string, parte_4: string]

In [15]:
clean_df = interrup.dropna(how='any', subset=[
    'IdeConjuntoUnidadeConsumidora', 'DscConjuntoUnidadeConsumidora', 'DscSubestacaoDistribuicao',
    'NumOrdemInterrupcao', 'DscTipoInterrupcao', 'IdeMotivoInterrupcao', 'DatInicioInterrupcao',
    'DatFimInterrupcao', 'DscFatoGeradorInterrupcao', 'NumNivelTensao', 'NumUnidadeConsumidora',
    'NumConsumidorConjunto', 'NumAno', 'NomAgenteRegulado', 'SigAgente', 'NumCPFCNPJ',
    'parte_1', 'parte_2', 'parte_3', 'parte_4'])

In [ ]:
interrup = interrup.fillna({'DscTipoInterrupcao': 'DESCONHECIDO'})

agg_df = interrup\
                .groupBy('DatInicioInterrupcao', 'NumOrdemInterrupcao')\
                .pivot('DscTipoInterrupcao')\
                .agg(f.lit(1))\
                .na.fill(0)

agg_df.show()

In [ ]:
data_interrup = agg_df.withColumnRenamed('DatInicioInterrupcao', 'DATA')

data_interrup.show()

In [ ]:
import pandas as pd

arquivo = '/content/drive/MyDrive/INMET_SE_SP_A771_SAO PAULO - INTERLAGOS_01-01-2023_A_31-12-2023.csv'
chuvas = pd.read_csv(arquivo, sep=';')
chuvas.head()


In [ ]:
df = pd.merge(data_interrup, chuvas, on='DATA', how='inner')

df.head()